# 4501 final project

## Project Setup

In [5]:
import os
import re
import pandas as pd
import numpy as np
import requests
import math
from bs4 import BeautifulSoup
import geopandas as gpd
import sqlite3
from sqlalchemy import create_engine
import matplotlib.pyplot as pltx
from urllib.parse import urljoin
import warnings
from tqdm import tqdm
warnings.simplefilter("ignore")

In [6]:

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "weather"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise


## Part 1: Data Preprocessing

### Download

Define a function download_file(link, save_dir) that efficiently download files ensuring that no duplicate downloads occur if the file already exists in the target directory and Saves the file in chunks of 8192 bytes to avoid memory overload.

In [7]:
# Local directory to save downloaded files
download_dir = "./nyc_taxi_data"
os.makedirs(download_dir, exist_ok=True)

# Function to download a file
def download_file(link, save_dir):
    file_name = link.split("/")[-1]
    file_path = os.path.join(save_dir, file_name)
    # Skip download if the file already exists
    if not os.path.exists(file_path):  
        print(f"Downloading {file_name}...")
        response = requests.get(link, stream=True)
        with open(file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Saved to {file_path}")
    else:
        print(f"{file_name} already exists. Skipping download.")
    return file_path

* Scraping a webpage for links to Yellow Taxi and HVFHV Parquet data files.
* Downloading only the relevant files based on naming patterns and storing them locally.

In [8]:
# Fetch the webpage content
response = requests.get(TLC_URL)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    raise Exception("Failed to fetch the TLC page.")

# Regular expressions to match Yellow Taxi and HVFHV links
yellow_taxi_regex = re.compile(r"yellow_tripdata_(202[0-4])-(0[1-9]|1[0-2])\.parquet", re.IGNORECASE)
hvfhv_regex = re.compile(r"fhvhv_tripdata_(202[0-4])-(0[1-9]|1[0-2])\.parquet", re.IGNORECASE)

# Find all links on the page
links = soup.find_all('a', href=True)

# File path
taxi_path = []
hvfhv_path = []

# Filter and download Yellow Taxi and HVFHV Parquet files
taxi_path, hvfhv_path = [], []
for link in soup.find_all("a", href=True):
    url = urljoin("https://www1.nyc.gov", link["href"].strip())
    if yellow_taxi_regex.search(url):
        taxi_path.append(download_file(url, download_dir))
    elif hvfhv_regex.search(url):
        hvfhv_path.append(download_file(url, download_dir))
                

yellow_tripdata_2024-01.parquet already exists. Skipping download.
fhvhv_tripdata_2024-01.parquet already exists. Skipping download.
yellow_tripdata_2024-02.parquet already exists. Skipping download.
fhvhv_tripdata_2024-02.parquet already exists. Skipping download.
yellow_tripdata_2024-03.parquet already exists. Skipping download.
fhvhv_tripdata_2024-03.parquet already exists. Skipping download.
yellow_tripdata_2024-04.parquet already exists. Skipping download.
fhvhv_tripdata_2024-04.parquet already exists. Skipping download.
yellow_tripdata_2024-05.parquet already exists. Skipping download.
fhvhv_tripdata_2024-05.parquet already exists. Skipping download.
yellow_tripdata_2024-06.parquet already exists. Skipping download.
fhvhv_tripdata_2024-06.parquet already exists. Skipping download.
yellow_tripdata_2024-07.parquet already exists. Skipping download.
fhvhv_tripdata_2024-07.parquet already exists. Skipping download.
yellow_tripdata_2024-08.parquet already exists. Skipping download.
fh

### Load Taxi Zones

* The first function (load_taxi_zones) processes and prepares taxi zone data from a shapefile for geospatial analysis.
* The second function (lookup_coords_for_taxi_zone_id) finds the latitude and longitude coordinates of a taxi zone based on its LocationID.

In [9]:
def load_taxi_zones(shapefile_path):
    taxi_zones = gpd.read_file(shapefile_path)
    taxi_zones = taxi_zones.to_crs(4326)  # Reproject to lat/lon (WGS84)
    taxi_zones['lon'] = taxi_zones.centroid.x  # Calculate longitude from centroid
    taxi_zones['lat'] = taxi_zones.centroid.y  # Calculate latitude from centroid
    return taxi_zones

In [10]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):
    zone = loaded_taxi_zones[loaded_taxi_zones['LocationID'] == zone_loc_id]
    if not zone.empty:
        return zone.iloc[0]['lat'], zone.iloc[0]['lon']
    return None, None  # Return None if the location ID is invalid

### Calculate Sample Size

Create a representative sample using Cochran's Sample Size Formula

In [11]:
def calculate_sample_size(population, margin_of_error=0.05):
    z = 1.96  # For 95% confidence
    p = 0.5  # Assumed proportion of 50% (worst-case scenario)
    q = 1 - p
    e = margin_of_error
    pop = population.shape[0]
    # Cochran's sample size formula
    sample_size = (z ** 2 * p * q) / (e ** 2)
    # Adjust sample size for finite population
    sample_size = sample_size / (1 + (sample_size - 1) / pop)
    sampled_df = population.sample(n=math.ceil(sample_size), random_state=42)
    return sampled_df

### Common Functions

Append latitude and longitude coordinates for pickup and drop-off locations to taxi_data DataFrame by mapping them from a preloaded taxi zones dataset.

In [12]:
def add_coordinates_to_taxi_data(taxi_data, loaded_taxi_zones):
    taxi_data[['PUlat', 'PUlon']] = taxi_data['PULocationID'].apply(
        lambda x: pd.Series(lookup_coords_for_taxi_zone_id(x, loaded_taxi_zones))
    )
    taxi_data[['DOlat', 'DOlon']] = taxi_data['DOLocationID'].apply(
        lambda x: pd.Series(lookup_coords_for_taxi_zone_id(x, loaded_taxi_zones))
    )
    taxi_data = taxi_data.drop(columns=['PULocationID', 'DOLocationID'])
    return taxi_data

### Process Taxi Data

1. Load Data: Reads a Parquet file specified by path into a pandas DataFrame using pd.read_parquet().

2. Initial Filtering:
 - Removes trips with zero or negative distances (trip_distance > 0).
 - Excludes trips where the pickup and drop-off locations are the same (PULocationID != DOLocationID).
 - Ensures the PULocationID and DOLocationID values are within the valid range of 1–263 .

3. Handle Missing Data: Drops rows with missing values in the trip_distance, PULocationID, or DOLocationID columns.

4. Sampling: Calculates the required sample size using calculate_sample_size() and selects a random sample from the dataset.

5. Add Geographic Coordinates: Enriches the dataset by adding latitude and longitude columns for both pickup and drop-off locations using the add_coordinates_to_taxi_data() function.

6. Filter by Geographic Boundaries: Filters trips to ensure pickup and drop-off locations are within specific latitude and longitude ranges.

7. Remove Unnecessary Columns: Drops columns that are not relevant for further analysis, such as: RatecodeID, store_and_fwd_flag, payment_type, extra


In [13]:
def get_and_clean_month(path, loaded_taxi_zones):
    a = pd.read_parquet(path)
    a = a[(a['trip_distance'] > 0) & (a['PULocationID'] != a['DOLocationID'])]
    # Ensures the PULocationID and DOLocationID values are within the valid range of 1–263
    a = a[(a['PULocationID'] >= 1) & (a['PULocationID'] <= 263)]
    a = a[(a['DOLocationID'] >= 1) & (a['DOLocationID'] <= 263)]
    # drop nan
    a = a.dropna(subset=['trip_distance', 'PULocationID', 'DOLocationID'])  
    # get sample
    a = calculate_sample_size(a)
    # Convert ID to lat lon
    a = add_coordinates_to_taxi_data(a, loaded_taxi_zones)
    # Filter by latitude
    a = a[a['PUlat'].between(40.560445, 40.908524)] 
    a = a[a['DOlat'].between(40.560445, 40.908524)] 
    # Filter by longitude
    a = a[a['PUlon'].between(-74.242330, -73.717047)] 
    a = a[a['DOlon'].between(-74.242330, -73.717047)]  
    a = a.drop(['RatecodeID', 'store_and_fwd_flag', 'payment_type', 
             'extra'], axis=1)
    # more clean step
    return a

* Cleaning each file individually.
* Merging all the cleaned files into a single DataFrame.

In [14]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    loaded_taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    for parquet_url in tqdm(parquet_urls):
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url, loaded_taxi_zones)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        all_taxi_dataframes.append(dataframe.sort_values(by='tpep_pickup_datetime', ascending=True))  
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes, ignore_index=True)
    return taxi_data

Return a cleaned and combined dataset of taxi trip data

In [15]:
def get_taxi_data():
    taxi_path.sort()
    taxi_data = get_and_clean_taxi_data(taxi_path)
    return taxi_data

In [17]:
taxi_data = get_taxi_data()

100%|██████████| 57/57 [00:47<00:00,  1.21it/s]


In [18]:
taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,PUlat,PUlon,DOlat,DOlon,Airport_fee
0,1,2020-01-01 00:56:39,2020-01-01 01:21:13,2.0,3.80,16.5,0.5,0.00,0.0,0.3,20.30,2.5,NaN,40.756729,-73.965146,40.712459,-73.998151,NaN
1,1,2020-01-01 02:09:23,2020-01-01 02:24:08,3.0,3.30,13.0,0.5,3.35,0.0,0.3,20.15,2.5,NaN,40.736824,-73.984052,40.775932,-73.946510,NaN
2,2,2020-01-01 02:35:25,2020-01-01 03:06:56,1.0,6.82,25.5,0.5,5.86,0.0,0.3,35.16,2.5,NaN,40.756729,-73.965146,40.761493,-73.919694,NaN
3,2,2020-01-01 03:11:10,2020-01-01 03:23:52,6.0,6.91,21.0,0.5,4.96,0.0,0.3,29.76,2.5,NaN,40.766238,-73.995135,40.841709,-73.941399,NaN
4,1,2020-01-01 03:49:45,2020-01-01 03:59:46,2.0,1.40,8.5,0.5,0.00,0.0,0.3,9.80,0.0,NaN,40.804334,-73.951292,40.818258,-73.940772,NaN


Adjust columns' names

In [22]:
new_column_names = {
    'VendorID': 'vendor_id',
    'tpep_pickup_datetime': 'pickup_time',
    'tpep_dropoff_datetime': 'dropoff_time',
    'passenger_count': 'passenger_count',
    'trip_distance': 'trip_distance',
    'fare_amount': 'fare_amount',
    'mta_tax': 'tax',
    'tip_amount': 'tip',
    'tolls_amount': 'tolls',
    'improvement_surcharge' : 'imp_surcharge',
    'total_amount': 'total_amount',
    'congestion_surcharge':'con_surcharge',
    'airport_fee': 'airport_fee',
    'PUlat':'pickup_lat',
    'PUlon':'pickup_lon',
    'DOlat':'dropoff_lat',
    'DOlon':'dropoff_lon',
    'Airport_fee':'fee'
}

taxi_data = taxi_data.rename(columns=new_column_names)

In [23]:
taxi_data.head()

,vendor_id,pickup_time,dropoff_time,passenger_count,trip_distance,fare_amount,tax,tip,tolls,imp_surcharge,total_amount,con_surcharge,airport_fee,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon
0,1,2020-01-01 00:56:39,2020-01-01 01:21:13,2.0,3.80,16.5,0.5,0.00,0.0,0.3,20.30,2.5,NaN,40.756729,-73.965146,40.712459,-73.998151
1,1,2020-01-01 02:09:23,2020-01-01 02:24:08,3.0,3.30,13.0,0.5,3.35,0.0,0.3,20.15,2.5,NaN,40.736824,-73.984052,40.775932,-73.946510
2,2,2020-01-01 02:35:25,2020-01-01 03:06:56,1.0,6.82,25.5,0.5,5.86,0.0,0.3,35.16,2.5,NaN,40.756729,-73.965146,40.761493,-73.919694
3,2,2020-01-01 03:11:10,2020-01-01 03:23:52,6.0,6.91,21.0,0.5,4.96,0.0,0.3,29.76,2.5,NaN,40.766238,-73.995135,40.841709,-73.941399
4,1,2020-01-01 03:49:45,2020-01-01 03:59:46,2.0,1.40,8.5,0.5,0.00,0.0,0.3,9.80,0.0,NaN,40.804334,-73.951292,40.818258,-73.940772


In [24]:
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21934 entries, 0 to 21933
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   vendor_id        21934 non-null  int64         
 1   pickup_time      21934 non-null  datetime64[ns]
 2   dropoff_time     21934 non-null  datetime64[ns]
 3   passenger_count  20795 non-null  float64       
 4   trip_distance    21934 non-null  float64       
 5   fare_amount      21934 non-null  float64       
 6   tax              21934 non-null  float64       
 7   tip              21934 non-null  float64       
 8   tolls            21934 non-null  float64       
 9   imp_surcharge    21934 non-null  float64       
 10  total_amount     21934 non-null  float64       
 11  con_surcharge    20795 non-null  float64       
 12  airport_fee      8187 non-null   float64       
 13  pickup_lat       21934 non-null  float64       
 14  pickup_lon       21934 non-null  float

In [25]:
taxi_data.describe()

,vendor_id,passenger_count,trip_distance,fare_amount,tax,tip,tolls,imp_surcharge,total_amount,con_surcharge,airport_fee,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon
count,21934.000000,20795.000000,21934.000000,21934.000000,21934.000000,21934.000000,21934.000000,21934.000000,21934.000000,20795.000000,8187.000000,21934.000000,21934.000000,21934.000000,21934.000000
mean,1.718291,1.396778,4.616993,15.519664,0.492523,2.738133,0.448342,0.552694,22.656455,2.300313,0.083975,40.752857,-73.967122,40.755038,-73.970701
std,0.449842,0.976673,164.542840,13.960149,0.079543,3.138831,1.856061,0.352648,17.752309,0.721166,0.316574,0.032189,0.045494,0.033799,0.036873
min,1.000000,0.000000,0.010000,-145.500000,-0.500000,-23.920000,-15.380000,-1.000000,-146.500000,-2.500000,-1.250000,40.576961,-74.170887,40.576961,-74.174000
25%,1.000000,1.000000,1.150000,7.500000,0.500000,0.000000,0.000000,0.300000,12.960000,2.500000,0.000000,40.740337,-73.989845,40.740337,-73.989845
50%,2.000000,1.000000,1.870000,11.000000,0.500000,2.200000,0.000000,0.300000,17.160000,2.500000,0.000000,40.756729,-73.977698,40.758028,-73.977698
75%,2.000000,1.000000,3.500000,17.700000,0.500000,3.500000,0.000000,1.000000,25.185000,2.500000,0.000000,40.773633,-73.965146,40.775932,-73.959635
max,2.000000,6.000000,24137.690000,172.400000,0.500000,50.000000,46.150000,1.000000,187.780000,2.500000,1.250000,40.897932,-73.735554,40.899529,-73.726655


### Processing Uber Data

1. Load Data: Reads the Parquet file from the specified url into a pandas DataFrame.

2. Filter by License: Retains only trips associated with hvfhs_license_num == 'HV0003' (representing Uber data).

3. Initial Filtering: Excludes trips with non-positive trip distances.
 - Removes trips where the pickup (PULocationID) and drop-off (DOLocationID) locations are the same.
 - Retains only rows where PULocationID and DOLocationID are within the valid range of 1–263.

4. Handle Missing Data: Drops rows with missing values in trip_distance, PULocationID, or DOLocationID.

5. Sampling: Reduces the dataset size by calculating and selecting an appropriate sample using calculate_sample_size().

6. Add Geographic Coordinates: Enriches the dataset by adding latitude and longitude columns for both pickup and drop-off locations using the add_coordinates_to_taxi_data() function.

7. Filter by Geographic Boundaries: Ensures that the latitude and longitude of both pickup and drop-off locations fall within specified geographic boundaries.

8. Drop Unnecessary Columns: Removes columns irrelevant for further analysis:  driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag, bcf, airport_fee.

In [26]:
def get_and_clean_uber_month(url, loaded_taxi_zones):
    # uber
    a = pd.read_parquet(url)
    a = a[(a['hvfhs_license_num'] == 'HV0003')]
    a = a.rename(columns={'trip_miles': 'trip_distance'})
    a = a[(a['trip_distance'] > 0) & (a['PULocationID'] != a['DOLocationID'])]
    a = a[(a['PULocationID'] >= 1) & (a['PULocationID'] <= 263)]
    a = a[(a['DOLocationID'] >= 1) & (a['DOLocationID'] <= 263)]
    a = a.dropna(subset=['trip_distance', 'PULocationID', 'DOLocationID'])  
    a = calculate_sample_size(a)
    a = add_coordinates_to_taxi_data(a, loaded_taxi_zones)
    # Filter by latitude
    a = a[a['PUlat'].between(40.560445, 40.908524)] 
    a = a[a['DOlat'].between(40.560445, 40.908524)] 
    # Filter by longitude
    a = a[a['PUlon'].between(-74.242330, -73.717047)] 
    a = a[a['DOlon'].between(-74.242330, -73.717047)]  
    a = a.drop([ 'driver_pay', 
            'shared_request_flag', 'shared_match_flag', 
            'access_a_ride_flag', 'wav_request_flag', 
            'wav_match_flag','bcf','airport_fee'], axis=1)
    return a

Processes and consolidates Uber trip data from multiple Parquet files into a single cleaned dataset.

In [27]:
def get_and_clean_uber_data(parquet_urls):
    all_uber_dataframes = []
    loaded_taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
    for parquet_url in tqdm(parquet_urls):
        dataframe = get_and_clean_uber_month(parquet_url, loaded_taxi_zones)
        all_uber_dataframes.append(dataframe.sort_values(by='pickup_datetime', ascending=True))  
    # create one gigantic dataframe with data from every month needed
    uber_data = pd.concat(all_uber_dataframes, ignore_index=True)
    return uber_data

In [28]:
def get_uber_data():
    hvfhv_path.sort()
    taxi_data = get_and_clean_uber_data(hvfhv_path)
    return taxi_data

In [30]:
uber_data = get_uber_data()

100%|██████████| 57/57 [08:35<00:00,  9.05s/it]


In [31]:
uber_data.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,trip_distance,trip_time,base_passenger_fare,tolls,sales_tax,congestion_surcharge,tips,PUlat,PUlon,DOlat,DOlon
0,HV0003,B02872,B02872,2020-01-01 00:35:28,2020-01-01 00:40:04,2020-01-01 00:42:54,2020-01-01 00:58:30,2.01,936,18.67,0.00,1.66,0.00,0.0,40.658744,-73.947442,40.652365,-73.922251
1,HV0003,B02889,B02889,2020-01-01 04:20:12,2020-01-01 04:20:47,2020-01-01 04:25:07,2020-01-01 04:44:12,4.71,1145,36.90,0.00,3.27,2.75,0.0,40.718938,-73.990896,40.715370,-73.936793
2,HV0003,B02836,B02836,2020-01-01 04:45:28,2020-01-01 04:48:10,2020-01-01 04:49:22,2020-01-01 05:08:37,4.07,1155,8.20,0.00,0.73,0.00,0.0,40.674469,-73.939287,40.695338,-73.986086
3,HV0003,B02835,B02835,2020-01-01 06:15:30,2020-01-01 06:19:21,2020-01-01 06:20:19,2020-01-01 06:29:43,1.94,564,10.51,0.00,0.93,0.00,0.0,40.652365,-73.922251,40.674469,-73.939287
4,HV0003,B02883,B02883,2020-01-01 16:06:43,2020-01-01 16:11:01,2020-01-01 16:11:14,2020-01-01 16:47:16,17.51,2162,76.34,6.12,7.32,2.75,0.0,40.758028,-73.977698,40.646985,-73.786533


In [32]:
new_column_names = {
    'hvfhs_license_num': 'license_number',
    'dispatching_base_num': 'dispatch_base',
    'originating_base_num': 'origin_base',
    'request_datetime': 'request_time',
    'on_scene_datetime': 'on_scene_time',
    'pickup_datetime': 'pickup_time',
    'dropoff_datetime': 'dropoff_time',
    'trip_distance': 'trip_distance',
    'trip_time': 'trip_duration',
    'base_passenger_fare':'fare_amount',
    'tolls': 'tolls',
    'sales_tax': 'tax',
    'congestion_surcharge': 'con_surcharge',
    'tips': 'tips',
    'PUlat': 'pickup_lat',
    'PUlon': 'pickup_lon',
    'DOlat': 'dropoff_lat',
    'DOlon': 'dropoff_lon'
}

uber_data = uber_data.rename(columns=new_column_names)

In [33]:
uber_data.head()

,license_number,dispatch_base,origin_base,request_time,on_scene_time,pickup_time,dropoff_time,trip_distance,trip_duration,fare_amount,tolls,tax,con_surcharge,tips,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon
0,HV0003,B02872,B02872,2020-01-01 00:35:28,2020-01-01 00:40:04,2020-01-01 00:42:54,2020-01-01 00:58:30,2.01,936,18.67,0.00,1.66,0.00,0.0,40.658744,-73.947442,40.652365,-73.922251
1,HV0003,B02889,B02889,2020-01-01 04:20:12,2020-01-01 04:20:47,2020-01-01 04:25:07,2020-01-01 04:44:12,4.71,1145,36.90,0.00,3.27,2.75,0.0,40.718938,-73.990896,40.715370,-73.936793
2,HV0003,B02836,B02836,2020-01-01 04:45:28,2020-01-01 04:48:10,2020-01-01 04:49:22,2020-01-01 05:08:37,4.07,1155,8.20,0.00,0.73,0.00,0.0,40.674469,-73.939287,40.695338,-73.986086
3,HV0003,B02835,B02835,2020-01-01 06:15:30,2020-01-01 06:19:21,2020-01-01 06:20:19,2020-01-01 06:29:43,1.94,564,10.51,0.00,0.93,0.00,0.0,40.652365,-73.922251,40.674469,-73.939287
4,HV0003,B02883,B02883,2020-01-01 16:06:43,2020-01-01 16:11:01,2020-01-01 16:11:14,2020-01-01 16:47:16,17.51,2162,76.34,6.12,7.32,2.75,0.0,40.758028,-73.977698,40.646985,-73.786533


In [34]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21851 entries, 0 to 21850
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   license_number  21851 non-null  object        
 1   dispatch_base   21851 non-null  object        
 2   origin_base     21846 non-null  object        
 3   request_time    21851 non-null  datetime64[ns]
 4   on_scene_time   21851 non-null  datetime64[ns]
 5   pickup_time     21851 non-null  datetime64[ns]
 6   dropoff_time    21851 non-null  datetime64[ns]
 7   trip_distance   21851 non-null  float64       
 8   trip_duration   21851 non-null  int64         
 9   fare_amount     21851 non-null  float64       
 10  tolls           21851 non-null  float64       
 11  tax             21851 non-null  float64       
 12  con_surcharge   21851 non-null  float64       
 13  tips            21851 non-null  float64       
 14  pickup_lat      21851 non-null  float64       
 15  pi

In [35]:
uber_data.describe()

,trip_distance,trip_duration,fare_amount,tolls,tax,con_surcharge,tips,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon
count,21851.000000,21851.000000,21851.000000,21851.000000,21851.000000,21851.000000,21851.000000,21851.000000,21851.000000,21851.000000,21851.000000
mean,4.651595,1120.405473,21.971942,0.714933,1.964367,1.106185,0.846985,40.739567,-73.936109,40.739079,-73.936559
std,4.370277,729.895150,15.577976,2.736384,1.437378,1.343959,2.539146,0.067755,0.063597,0.068204,0.066896
min,0.210000,68.000000,-15.760000,0.000000,0.000000,0.000000,0.000000,40.561994,-74.170887,40.561994,-74.174000
25%,1.730000,614.500000,11.540000,0.000000,1.000000,0.000000,0.000000,40.694428,-73.985156,40.691831,-73.984196
50%,3.060000,932.000000,17.690000,0.000000,1.560000,0.000000,0.000000,40.739496,-73.949540,40.740337,-73.948891
75%,5.920000,1419.000000,27.225000,0.000000,2.450000,2.750000,0.000000,40.775965,-73.900316,40.775965,-73.899735
max,37.470000,8173.000000,204.350000,51.350000,17.190000,2.750000,43.670000,40.899529,-73.726655,40.899529,-73.726655


### Processing Weather Data

Retrieves and sorts all weather CSV files, filtering for files associated with the years 2020 to 2024.

In [36]:
def get_all_weather_csvs(directory):
    # List to store file paths for all weather CSVs from 2020 to 2024
    weather_files = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            for year in range(2020, 2025):
                if str(year) in filename:
                    weather_files.append(os.path.join(directory, filename))  
    weather_files.sort()
    return weather_files

Processes a group of precipitation data, handling special cases like "trace" values ('T') and summing numeric values while ensuring robust handling of non-numeric or missing data.

In [37]:
def process_precipitation(group):
        def safe_float_conversion(x):
            try:
                return float(x)
            except ValueError:
                return 0    
        if all(x == 'T' for x in group):
            return 'T'       
        elif 'T' in group.values:
            numeric_sum = sum(safe_float_conversion(x) for x in group if x not in ['T', np.nan])
            return numeric_sum if numeric_sum > 0 else 'T'       
        else:
            return sum(safe_float_conversion(x) for x in group if pd.notna(x))

Processes hourly weather data from a CSV file, cleans it, and ensures it is aggregated, consistent, and complete for time-series analysis.

In [38]:
def clean_month_weather_data_hourly(csv_file):
    df = pd.read_csv(csv_file)
    # choose column
    hourly_column = ['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']
    df_tmp = df[hourly_column]
    hourly_data = df_tmp[~df_tmp['DATE'].str.contains("T23:59:00", na=False)]
    hourly_data['DATE'] = pd.to_datetime(hourly_data['DATE'])
   
    hourly_data['DATE'] = pd.to_datetime(hourly_data['DATE'])
    hourly_data['HourGroup'] = hourly_data['DATE'].dt.floor('H')
    hourly_data['HourlyPrecipitation'] = hourly_data['HourlyPrecipitation'].replace(np.nan, 0)
    
    
    aggregated = hourly_data.groupby('HourGroup').agg(
        HourlyPrecipitation=('HourlyPrecipitation', process_precipitation),
        HourlyWindSpeed=('HourlyWindSpeed', lambda x: x.mean() if len(x) > 0 else 0)
    ).reset_index()
    
    
    aggregated['DATE'] = aggregated['HourGroup'] + pd.Timedelta(minutes=51)
    aggregated = aggregated.drop(columns=['HourGroup'])
    aggregated = aggregated[['DATE'] + [col for col in aggregated.columns if col != 'DATE']]
    
    
    year = aggregated['DATE'].dt.year.min()  
    max_date = aggregated['DATE'].max()
    start_date = f"{year}-01-01 23:51:00"   
    end_date = max_date    
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='H')
    missing_dates = set(full_date_range) - set(aggregated['DATE'])
    if missing_dates: 
        for missing_date in sorted(missing_dates):  
            prev_hour = missing_date - pd.Timedelta(hours=1)
            if prev_hour in set(aggregated['DATE']):
                prev_data = aggregated[aggregated['DATE'] == prev_hour].iloc[0].copy()
                prev_data['DATE'] = missing_date
                aggregated = pd.concat([aggregated, pd.DataFrame([prev_data])], ignore_index=True)
    hourly_data = aggregated.sort_values(by='DATE').reset_index(drop=True)
    return hourly_data

Processes daily weather data from a CSV file, ensures its completeness, and imputes missing data by leveraging hourly precipitation data.

In [39]:
def clean_month_weather_data_daily(csv_file,hourly_data):
    df = pd.read_csv(csv_file)
    daily_column = ['DATE', 'DailyAverageWindSpeed', 
                'DailyPrecipitation', 'DailySnowfall', 
                'DailySustainedWindSpeed', 'DailyWeather']
    df_tmp = df[daily_column]
    daily_data = df_tmp[df_tmp['DATE'].str.contains("T23:59:00", na=False)]
    daily_data = daily_data.groupby('DATE', as_index=False).first()
    daily_data['DATE'] = pd.to_datetime(daily_data['DATE'])
    
    if daily_data.shape[0] != 365 and daily_data.shape[0] != 366 and 360 <= daily_data.shape[0]:
        year = daily_data['DATE'].dt.year.min()  
        start_date = f"{year}-01-01 23:59:00"
        end_date = f"{year}-12-31 23:59:00"
        full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
        missing_dates = set(full_date_range) - set(daily_data['DATE'])
        
        if missing_dates:
            new_rows = []
            for missing_date in sorted(missing_dates):
                previous_valid_date = daily_data[daily_data['DATE'] < missing_date].iloc[-1]
                new_row = previous_valid_date.copy()
                new_row['DATE'] = missing_date
    
                precipitation_sum = hourly_data[hourly_data['DATE'].dt.date == missing_date.date()]['HourlyPrecipitation'].sum()
                new_row['DailyPrecipitation'] = precipitation_sum
                new_rows.append(new_row)
            daily_data = pd.concat([daily_data, pd.DataFrame(new_rows)], ignore_index=True)
            daily_data.sort_values(by='DATE', inplace=True)
    return daily_data

Return two DataFrames:

hourly_data: Contains consolidated hourly weather data.
daily_data: Contains consolidated daily weather data.

In [40]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file, hourly_dataframe)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [41]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [42]:
hourly_weather_data.head()

,DATE,HourlyPrecipitation,HourlyWindSpeed
0,2020-01-01 00:51:00,0.0,8.0
1,2020-01-01 01:51:00,0.0,8.0
2,2020-01-01 02:51:00,0.0,14.0
3,2020-01-01 03:51:00,0.0,11.0
4,2020-01-01 04:51:00,0.0,6.0


In [43]:
new_column_names = {
    'DATE': 'datetime',
    'HourlyPrecipitation': 'precipitation',
    'HourlyWindSpeed': 'windspeed'
}

hourly_weather_data = hourly_weather_data.rename(columns=new_column_names)

In [44]:
hourly_weather_data.head()

,datetime,precipitation,windspeed
0,2020-01-01 00:51:00,0.0,8.0
1,2020-01-01 01:51:00,0.0,8.0
2,2020-01-01 02:51:00,0.0,14.0
3,2020-01-01 03:51:00,0.0,11.0
4,2020-01-01 04:51:00,0.0,6.0


In [45]:
hourly_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42163 entries, 0 to 7098
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime       42163 non-null  datetime64[ns]
 1   precipitation  42163 non-null  object        
 2   windspeed      38712 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.3+ MB


In [46]:
hourly_weather_data.describe()

,windspeed
count,38712.000000
mean,5.103946
std,16.449381
min,0.000000
25%,3.000000
50%,5.000000
75%,7.000000
max,2237.000000


In [47]:
daily_weather_data.head()

,DATE,DailyAverageWindSpeed,DailyPrecipitation,DailySnowfall,DailySustainedWindSpeed,DailyWeather
0,2020-01-01 23:59:00,8.6,0.00,0.0,17.0,None
1,2020-01-02 23:59:00,5.4,0.00,0.0,13.0,None
2,2020-01-03 23:59:00,3.4,0.15,0.0,10.0,RA BR HZ
3,2020-01-04 23:59:00,4.4,0.27,0.0,15.0,RA FG BR
4,2020-01-05 23:59:00,11.3,T,0.0,25.0,RA


In [48]:
new_column_names = {
    'DATE': 'datetime',
    'DailyAverageWindSpeed': 'avg_windspeed',
    'DailyPrecipitation': 'precipitation',
    'DailySnowfall': 'snowfall',
    'DailySustainedWindSpeed': 'sustained_windspeed',
    'DailyWeather': 'daily_weather'
}

daily_weather_data = daily_weather_data.rename(columns=new_column_names)

In [49]:
daily_weather_data.head()

,datetime,avg_windspeed,precipitation,snowfall,sustained_windspeed,daily_weather
0,2020-01-01 23:59:00,8.6,0.00,0.0,17.0,None
1,2020-01-02 23:59:00,5.4,0.00,0.0,13.0,None
2,2020-01-03 23:59:00,3.4,0.15,0.0,10.0,RA BR HZ
3,2020-01-04 23:59:00,4.4,0.27,0.0,15.0,RA FG BR
4,2020-01-05 23:59:00,11.3,T,0.0,25.0,RA


In [50]:
daily_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1756 entries, 0 to 294
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   datetime             1756 non-null   datetime64[ns]
 1   avg_windspeed        1698 non-null   float64       
 2   precipitation        1756 non-null   object        
 3   snowfall             1751 non-null   object        
 4   sustained_windspeed  1756 non-null   object        
 5   daily_weather        831 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 96.0+ KB


In [51]:
daily_weather_data.describe()

,avg_windspeed
count,1698.000000
mean,5.001590
std,2.338815
min,0.600000
25%,3.200000
50%,4.600000
75%,6.375000
max,14.200000


## Part 2: Storing Cleaned Data

### Add Data to Database

## Part 3: Understanding the Data

In [67]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [68]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [69]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

NameError: name 'engine' is not defined

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)